In [1]:
from fyp_work.paper_to_equation.Gemini.GeminiPredict import GeminiPredict
from sympy import *
import numpy as np

In [2]:
api_key = "GEMINI_API_KEY"
filename = "Data/HTC_gemini4"
model = "tunedModels/mmltopython4-f3fuppiemnq9"

gp = GeminiPredict(api_key, filename, model)
gp.predict_from_file(filename="Data/HTC_mml") 
gp.save_predictions()

API key loaded successfully!


Generating Predictions:   0%|          | 0/13 [00:00<?, ?row/s]

Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.53s/row]


In [8]:
class SystemOfEquations():
    def __init__(self, filepath):
        self.equations = []
        self.eq = []
        self.filepath = filepath
        self.symbol_lines = set() # Only keep one instance of each symbol line
        self.symbols = set() # Same for symbols

    def extract_equations(self):
        with open (f"{self.filepath}", "r") as file:
            for line in file:
                self.equations.append(line)

    def get_equations(self):
        if not self.equations:
            self.extract_equations()

        return self.equations

    def validation_check(self, equation):
        try:
            eval(equation)
            return True
        except Exception as e:
            print(e)
            return False

    def parse_equation(self, equation):
        for line in equation.split("\\n"):
            if "Symbol" in line:
                line = line.replace('"', "")
                self.symbol_lines.add(line)
                self.symbols.add(line.split(" = ")[0])
            elif "Eq" in line:
                eq = line.split(" = ")[1]
                self.eq.append(eq)
        
    def solve(self, target):
        pass
    
    def solve_test(self, target):
        symbol_list = {"a", "b", "c"}
        const = {"c"}

        # Define symbols dynamically
        sd = {name: Symbol(name) for name in symbol_list if name not in const}

        # Substitute c = 4 directly in the equations
        c = 4
        # og_eq = [Eq(a, b + 5), Eq(b, c + 1)]
        equations = [Eq(sd["a"], sd["b"] + 5), Eq(sd["b"], c + 1)]
        # equations = [eq.subs({a: sd["a"], b: sd["b"]}) for eq in og_eq]

        # Solve for a
        result = solve(equations)
        print(result[sd[target]])
    
    def solve_test2(self, target):
        pass






def main():
    soe = SystemOfEquations("Data/HTC_gemini5_predictions.txt")
    soe.extract_equations()
    for eq in soe.equations:
        soe.parse_equation(eq)
        print(soe.validation_check(eq))
    # soe.solve_test("a")
    


main()

True
True
True
True
True
True
True
True
True
True
True
True
True
